In [12]:
!pip install pandas
!pip install faker
!pip install pymongo
!pip install matplotlib
!pip install streamlit

In [36]:
import pandas as pd
from faker import Faker
import pymongo
import matplotlib.pyplot as plt
import streamlit as st
import random
import datetime
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
from config import uri_mongo
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
client = MongoClient(uri_mongo,server_api=ServerApi('1'))
!ls -a

config.py  .gitignore  ml_project.ipynb


In [27]:
fake=Faker()
db=client["ecommerce"]
clients_col=db["clients"]
commandes_col=db["commandes"]

In [28]:
clients_col.delete_many({})
commandes_col.delete_many({})

DeleteResult({'n': 11042, 'electionId': ObjectId('7fffffff0000000000000038'), 'opTime': {'ts': Timestamp(1745246302, 512), 't': 56}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1745246302, 512), 'signature': {'hash': b':{\xbf\x96q\x9a\x89\xbe\x83\x93\xb1o\xcd.\xdf\x8eDVf\x97', 'keyId': 7464295776219299841}}, 'operationTime': Timestamp(1745246302, 512)}, acknowledged=True)

In [29]:
#Générer les produits
produits=[
    {"nom":"T-shirt","catégorie":"Vêtements","prix":20},
    {"nom":"Jean","catégorie":"Vêtements","prix":45},
    {"nom":"Basket","catégorie":"Chaussures","prix":70},
    {"nom":"Bracelet","catégorie":"Accessoires","prix":18},
    {"nom":"Collier","catégorie":"Accessoires","prix":25},
    {"nom":"Sac à main", "catégorie":"Bagagerie", "prix":20},
    {"nom":"Sac à dos", "catégorie":"Bagagerie", "prix":30},
    {"nom":"Montre", "catégorie":"Accessoires", "prix":90},
    {"nom":"Full Korean skincare", "catégorie":"Beauté", "prix":120},
    {"nom":"Chargeur", "catégorie":"Informatique","prix":15},

]

In [31]:
clients=[]
for i in range(100):
  client={
  "nom":fake.last_name(),
  "prenom":fake.first_name(),
  "adresse":fake.address(),
  "ville":fake.city(),
  "code_postal":fake.postcode(),
  "pays":fake.country(),
  "email":fake.email(),
  "telephone":fake.phone_number(),
  }
client_id=clients_col.insert_one(client).inserted_id
clients.append({"_id": client_id, **client})

In [46]:
commandes=[]
moyen_paiement=["Carte bancaire", "Paypal", "Virement", "Apple Pay"]
code_promo=[None,"WELCOME10","SALES20","FREESHIP"]
for i in range(2000):
  client=random.choice(clients)
  nb_produits=random.randint(1, len(produits))
  produit_commande=[]
  montant_totale=0
  for j in range(nb_produits):
    produit=random.choice(produits)
    quantite= random.randint(1,5)
    prix=produit["prix"]
    montant=round(prix*quantite,2)
    produit_commande.append({
        "nom":produit["nom"],
        "prix_unitaire":prix,
        "quantite":quantite,
        "montant":montant
    })
    montant_totale+=montant
    date=fake.date_between(start_date='-6M', end_date='today')
    commande={
        "client_id":client["_id"],
        "date":date.isoformat(),
        "email":client["email"],
        "pays":client["pays"],
        "produits":produit_commande,
        "moyen_paiement":random.choice(moyen_paiement),
        "code_promo":random.choice(code_promo),
        "montant_totale":montant_totale

    }
    commandes.append(commande)
for doc in commandes_col.find().limit(3):
   print(doc)


In [ ]:
#KPIs
#Chiffres d'affaires
pipeline = [
    {
        "$group": {
            "_id": "$pays",
            "chiffre_affaires": { "$sum": "$montant" },
            "nb_commandes": { "$sum": 1 }
        }
    },
    {
        "$sort": { "chiffre_affaires": -1 }
    }
]

resultat = list(commandes_col.aggregate(pipeline))

In [ ]:
#Top 10 clients

pipeline=[
    {
        "$group":{
            "_id":"$client_id",
            "montant_total":{"$sum":"$montant"},
            "nb_commandes":{"$sum":1}
        }
    },
    {
        "$sort":{"montant_total":-1}
    },
    {"$limit":10},
    {
        "$lookup":{
            "from":"clients",
            "localField":"_id",
            "foreignField":"_id",
            "as":"client"
    }},
    {"$unwind":"$client"},
    {"$project":{
        "nom":"$client.nom",
        "prenom":"$client.prenom",
        "email": "$client.email",
        "adresse":"$client.adresse",
        "ville":"$client.ville",
        "pays":"$client.pays",
        "chiffre_affaires":"$montant_total",
        "nb_commandes":"$nb_commandes"
    }}
]
top_clients=list(commandes_col.aggregate(pipeline))
for client in top_clients:
  print(f"{client['nom']} {client['prenom']} ({client['email']}) - {client['adresse']}, {client['ville']}, {client['pays']} - {round(client['chiffre_affaires'],2)} EUR sur {client['nb_commandes']} commandes.")


In [ ]:
#panier moyen
pipeline=[
    {
          "$group":{
              "_id":None,
              "total_montant":{"$sum":"$montant"},
              "total_commandes":{"$sum":1}
          }
      },
      {
          "$project":{
              "_id":0,
              "panier_moyen":{"$divide": ["$total_montant","$total_commandes"]
              },
              "total_montant":1,
              "total_commandes":1
          }
      }
]

resultat= list(commandes_col.aggregate(pipeline))[0]
print(f"Le panier moyen est : {round(resultat['panier_moyen'],2)} EUR")

In [ ]:
#panier moyen par mois

pipeline=[
    {
        "$group":{
            "_id": {"$substr":["$date",0,7]},
            "nb_commandes": {"$sum":1},
            "chiffre_affaires":{"$sum":"$montant"},
        }
    },
    {
        "$project":{
            "nb_commandes":1,
            "chiffre_affaires":1,
            "panier_moyen":{"$divide":["$chiffre_affaires","$nb_commandes"]}
        }
    },
    {
        "$sort":{"_id":1}
    }
]
resultat=list(commandes_col.aggregate(pipeline))
for ligne in resultat:
  print(f"{ligne['_id']} : {ligne['nb_commandes']} commandes - {round(ligne['chiffre_affaires'],2)} EUR - panier moyen {round(ligne['panier_moyen'],2)}")

#dataframe pour panier moyen et affichage

In [ ]:
#Segmentation des clients
pipeline=[
    {
        "$group":{
            "_id":"$client_id",
            "chiffre_affaires":{"$sum":"$montant"},
            "nb_commandes":{"$sum":1},
            "nb_commandes_promos":{
                "$sum":{
                    "$cond":[{"$ne":["$code_promo",None]},1,0]}} #si le code promo est diff de None alors compte 1 sinon 0
        }
    },
    {
        "$project":{
            "nb_commandes":1,
            "chiffre_affaires":1,
            "code_promo":1,
            "panier_moyen":{"$round":[{"$divide":["$chiffre_affaires","$nb_commandes"]},2]},
            "ratio_code_promos": {"$round":[{"$divide":["$nb_commandes_promos","$nb_commandes"]},2]
        }}
    },
    {
        "$lookup":{
            "from":"clients",
            "localField":"_id",
            "foreignField":"_id",
            "as":"client"
        }
    },
    {
        "$unwind":"$client"
    },
    #Affichage
    {
        "$project":{
            "client_id":"$_id",
            "nom":"$client.nom",
            "prenom":"$client.prenom",
            "pays":"$client.pays",
            "email":"$client.email",
            "chiffre_affaires":1,
            "nb_commandes":1,
            "panier_moyen":1,
            "ratio_code_promos":1
        }
    }
]

resultat=list(commandes_col.aggregate(pipeline))